In [1]:
import bz2
import numpy as np
import pandas as pd
import array
import gc 
import matplotlib.pyplot as plt
import time

In [2]:
def pulses(file_path):
    """
    Versión optimizada:
    - Usa bz2 (backend C) en lugar de subprocess.
    - Evita operaciones de string innecesarias.
    """
    data = []
    presion = 0.0
    clocks = 0
    current_pulse = []
    nl = 0
    cuentas = 0 
    with bz2.open(file_path, 'rt') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            if line.startswith('#'):
                if len(line) > 2 and line[2] == 't':
                    tval = int(line[6:])   
                    if tval<clocks:
                        tval+=125000000
                    
                    if nl == 32:
                        current_pulse.append((tval - clocks) * 8)
                        data.append(current_pulse)
        
                    clocks = int(line[6:])
                    nl = 0 
                    current_pulse = []
                
                if len(line) > 2 and line[2:4] == 'r1':
                    try:
                        valor = float(line.split()[2])
                        cuentas += valor
                    except (ValueError, IndexError):
                        pass
                elif len(line) > 5 and line[4] == 'p':
                    try:
                        valor = float(line.split()[3])
                        presion += valor
                    except (ValueError, IndexError):
                        pass
                continue

            if nl < 32:
                try:
                    current_pulse.append(int(line.split()[0]))
                except ValueError:
                    continue
            nl += 1

    if not data:
        return None, None

    df = pd.DataFrame(np.array(data, dtype=np.float32),
                      columns=[*range(32), "deltaTime"])
    presion /= 3600.0
    return df, presion,cuentas

In [3]:
# -------------------------
# 2. Funciones vectorizadas
# -------------------------

def BaseLine(df):
    num = abs(df.iloc[:, :32].min(axis=1))
    df.iloc[:, :32] = df.iloc[:, :32].add(num, axis=0)
    return df


def Peak_Position(df, sat=False, bin_sat_position=None):
    
    
    df['Peak_Position'] = df.iloc[:, :32].values.argmax(axis=1)
    if sat and bin_sat_position is not None:
        df = df[df['Peak_Position'] < bin_sat_position].reset_index(drop=True)
    
    df['Peak'] = df.iloc[:,0:32].max(axis=1)
    
    return df

def A3CAP(df):
    arr = df.iloc[:, :32].to_numpy(copy=False)
    peak = df['Peak_Position'].to_numpy()
    idx = np.arange(len(arr))
    mean_next3 = (arr[idx, peak + 1] + arr[idx, peak + 2] + arr[idx, peak + 3]) / 3
    df['A3CAP'] = ((arr[idx, peak] - mean_next3) * 100 / arr[idx, peak]).astype(np.int16)
    return df


def SPC(df, offset, name):
    arr = df.iloc[:, :32].to_numpy(copy=False)
    peak = df['Peak_Position'].to_numpy()
    idx = np.arange(len(arr))
    df[name] = ((arr[idx, peak] - arr[idx, peak + offset]) * 100 / arr[idx, peak]).astype(np.int16)
    return df


def charge(df):
    arr = df.iloc[:, :32].to_numpy(copy=False)
    df['Charge'] = arr.sum(axis=1)
    df = df[df['Charge'] >= 0].reset_index(drop=True)
    return df

In [4]:
def process_file(file_path, sat=False, bin_sat_position=None):
    df,presion,cuentas = pulses(file_path)
    #df = BaseLine(df)
    if df is not None:
        mask =  (df['deltaTime'] <2E7) & (df['deltaTime']>1000)
        df = df[mask]
        df = Peak_Position(df, sat, bin_sat_position)
        #df = A3CAP(df)
        df = SPC(df, 3, "SPC-24")
        #df = SPC(df, 4, "SPC-32")
        df = charge(df)
        mask_2 =  (df['SPC-24']<70) & (df['Charge']<15000)
        df = df[mask_2]
        return df,presion,cuentas
    else:
        return None

In [5]:
#def Charge_Histogram_All(df,df_1, df_4,cluster_col='Cluster', bin_width=7):
def Charge_Histogram_All(df,df_1, bin_width=7):
#def Charge_Histogram_All(df,cluster_col='Cluster',bin_width=7):
    """
    Generates an histogram from the column 'Charge':
    - Global all data
    - Splits the data according to each cluster.

    Parameters:
    - df: DataFrame with the column 'Charge' could be cluster_col (ej: 0,1,2)
    - cluster_col: name of the cluster
    - bin_width: the width of the bins
    """
    plt.figure(figsize=(15, 8))

    # --- bins compartidos para todos ---

    min_val, max_val = df['Charge'].min(), df['Charge'].max()
    bins = np.arange(min_val, max_val + bin_width, bin_width)

    # --- histograma global ---
    counts, bin_edges = np.histogram(df['Charge'], bins=bins)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    plt.semilogy(bin_centers, counts, color='black', lw=3,alpha=0.6, label="Raw", zorder=5)

    counts_1, bin_edges_1 = np.histogram(df_1['Charge'], bins=bins)
    bin_centers_1 = (bin_edges_1[:-1] + bin_edges_1[1:]) / 2
    plt.semilogy(bin_centers_1, counts_1, color='red', lw=3,alpha=0.3, label="Clean", zorder=5)

    #counts_4, bin_edges_4 = np.histogram(df_4['Charge'], bins=bins)
    #bin_centers_4 = (bin_edges_3[:-1] + bin_edges_3[1:]) / 2
    #plt.semilogy(bin_centers_4, counts_4, color='green', lw=3,alpha=0.3, label="Muons", zorder=5)

    # --- histogramas por cluster ---
    #clusters = sorted(df[cluster_col].unique())
    #for c in clusters:
        #subset = df[df[cluster_col] == c]['Charge'].to_numpy()
        #counts, bin_edges = np.histogram(subset, bins=bins)
        #bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        #plt.semilogy(bin_centers, counts, label=f"Cluster {c}", alpha=1, zorder=4)

    plt.ylabel("Number of Entries", fontsize=20)
    plt.xlabel("Charge[ADC]", fontsize=20)
    #plt.title("Charge Histogram", fontsize=20)
    plt.grid(True)
    plt.legend(fontsize='20')
    plt.show()

In [6]:
base_date = '2024_04'
base_path_data = '/eos/user/d/dmerizal/SWAN_projects/LAGO/Datos_ICRC/'
base_path_save = '/eos/user/d/dmerizal/SWAN_projects/LAGO/Datos/dias/'
#days = [d for d in range(1, 32) if d != 20 and d != 27 and d!=26]
# Lista de horas a procesar
days = [d for d in range(1, 2)]
hours = range(0,24)

start_time = time.time()
df_cleaned = pd.DataFrame([])
conteo = []
presiones  = []

for day in days:
    day_str = f"{day:02d}"
    output_csv_pulsos = f"{base_path_save}{base_date}_{day_str}.csv"
    output_csv_cuentas = f"{base_path_save}{base_date}_{day_str}_cuentas.csv"
    output_csv_presiones = f"{base_path_save}{base_date}_{day_str}_presiones.csv"
    
    for hour in hours:
        hour_str = f"{hour:02d}"
        file_path = f"{base_path_data}jaguarito_nogps_{base_date}_{day_str}_{hour_str}h00.dat.bz2"
        print(f"\nProcesando hora {hour_str}h y día {day_str}")
        
        df_reduced = None  # Por si falla el archivo
        
        try:
            df_reduced,presion,cuentas = process_file(file_path, sat=True, bin_sat_position=20)
            print(len(df_reduced))
            
            if df_reduced is not None and len(df_reduced) > 0:
                df_cleaned = pd.concat([df_cleaned, df_reduced], ignore_index=True)
                presiones.append(presion)
                conteo.append(cuentas)
                print(f"Archivo guardado para la hora {hour_str}h y día {day_str}")
        
        except Exception as e:
            print(f"❌ Error procesando {file_path}: {e}. Archivo omitido.")
            conteo.append(0)
            presiones.append(0)
            # df_reduced queda como None
        
        finally:
            # Liberar memoria
            if df_reduced is not None:
                del df_reduced
            gc.collect()
        
    print(f"Archivo final pulsos guardado en: {output_csv_pulsos}")
    df_cleaned.to_csv(output_csv_pulsos, index=False)
    
    print(f"Archivo final cuentas guardado en: {output_csv_cuentas}")
    df_cuentas = pd.DataFrame({'cuentas': conteo})
    df_cuentas.to_csv(output_csv_cuentas, index=False)
    
    print(f"Archivo final presiones guardado en: {output_csv_presiones}")
    df_cuentas = pd.DataFrame({'presion': presiones})
    df_cuentas.to_csv(output_csv_presiones, index=False)
    
        
        

# Guardar CSV final
#df_cleaned.to_csv(output_csv, index=False)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"\n✅ Procesamiento completo")
print(f"Tiempo total de ejecución: {elapsed_time:.2f} s")



Procesando hora 00h y día 01
177103
Archivo guardado para la hora 00h y día 01

Procesando hora 01h y día 01
176634
Archivo guardado para la hora 01h y día 01

Procesando hora 02h y día 01
176015
Archivo guardado para la hora 02h y día 01

Procesando hora 03h y día 01
176368
Archivo guardado para la hora 03h y día 01

Procesando hora 04h y día 01
179617
Archivo guardado para la hora 04h y día 01

Procesando hora 05h y día 01
180968
Archivo guardado para la hora 05h y día 01

Procesando hora 06h y día 01
178909
Archivo guardado para la hora 06h y día 01

Procesando hora 07h y día 01
181623
Archivo guardado para la hora 07h y día 01

Procesando hora 08h y día 01
183780
Archivo guardado para la hora 08h y día 01

Procesando hora 09h y día 01
187571
Archivo guardado para la hora 09h y día 01

Procesando hora 10h y día 01
181000
Archivo guardado para la hora 10h y día 01

Procesando hora 11h y día 01
176985
Archivo guardado para la hora 11h y día 01

Procesando hora 12h y día 01
174606
Arc

In [ ]:
#mask = (df['deltaTime']>1000) & (df['SPC-24']<80) 
#mask = (df['SPC-24']<70)
len(df[mask])

In [ ]:
mask_1 = (df['deltaTime']>1000) & (df['A3CAP']<70) & (df['Charge']<15000)
mask = (df['Charge']<15000)
Charge_Histogram_All(df[mask],df[mask_1], bin_width=7)

In [ ]:
mask_1 = (df['deltaTime']>1000) & (df['SPC-24']<70) & (df['Charge']<15000)
base_path_save = '/eos/user/d/dmerizal/SWAN_projects/LAGO/Datos/dias/'
output_csv = f"{base_path_save}reduced_data_{base_date}_Prueba.csv"
df_cleaned = df[mask_1]
df_cleaned.to_csv(output_csv, index=False)